In [14]:
using LinearAlgebra
using OMEinsum
using Test

# task 1
The enisum symbol of the outer product is given by
$$
i, j \to ij
$$
here is an example

In [5]:
ein"i, j -> ij"([1, 2, 3], [4, 5, 6])

3×3 Matrix{Int64}:
  4   5   6
  8  10  12
 12  15  18

#  task 2
The einsum notation $jk,kl,lm,mj \to $ stands for trace of multipulication of four matrices, given by
$$
\text{Tr}{(A B C D)} = \sum_{j, k, l, m} A_{jk} B_{kl} C_{lm} D_{mj}\;.
$$
Here is an example:

In [12]:
j, k, l, m = [rand(1:10) for i in 1:4]
A = rand(j, k)
B = rand(k, l)
C = rand(l, m)
D = rand(m, j)

@test ein"jk, kl, lm, mj ->"(A, B, C, D)[1] ≈ tr(A * B * C * D)

Test Passed

# task 3
The tensor network `"abc,cde,efg,ghi,ijk,klm,mno->abdfhjlno"` is known as matrix product state in physics or tensor train in mathematics. Please
    
1. Draw a diagramatic representation for it.
    
2. If we contract it with another tensor network `"pbq,qdr,rfs,sht,tju,ulv,vnw->pbdfhjlnw"`, i.e., computing `abc,cde,efg,ghi,ijk,klm,mno,pbq,qdr,rfs,sht,tju,ulv,vnw->apow`. What is the optimal contraction order in the diagram, and estimate the contraction complexity (degree of freedoms have the same size $n$).
    
3. Using `OMEinsum` (check the section "Probability graph") to obtain a contraction order and compare it with your answer.

![tensors](./tensors.jpg)

The optimal contraction order should be:

Step 1
$$
abc, pbq \to acpq\\
cde, pdr \to ceqr\\
......
$$
which will give us $7$ order-4 tensors, and time cost will be $7 \times n^5$.

Step 2
$$
acpq, cepr \to aepr\\
aepr, egrs \to agps\\
......\\
ampv, mpvw \to aopw
$$
and the cost will be $6 \times n^6$

The total cost will be $7 \times n^5 + 6 \times n^6$

In [17]:
eincode = ein"abc,cde,efg,ghi,ijk,klm,mno,pbq,qdr,rfs,sht,tju,ulv,vnw->apow"

abc, cde, efg, ghi, ijk, klm, mno, pbq, qdr, rfs, sht, tju, ulv, vnw -> apow

In [23]:
optimized_eincode = optimize_code(eincode, uniformsize(eincode, 2), TreeSA())

SlicedEinsum{Char, DynamicNestedEinsum{Char}}(Char[], apvm, vwmo -> apow
├─ kuap, uvkm -> apvm
│  ├─ iktu, tiap -> kuap
│  │  ├─ ijk, tju -> iktu
│  │  │  ├─ ijk
│  │  │  └─ tju
│  │  └─ ticq, acpq -> tiap
│  │     ├─ stgi, gcqs -> ticq
│  │     │  ├─ sht, ghi -> stgi
│  │     │  │  ⋮
│  │     │  │  
│  │     │  └─ fgcd, qdfs -> gcqs
│  │     │     ⋮
│  │     │     
│  │     └─ abc, pbq -> acpq
│  │        ├─ abc
│  │        └─ pbq
│  └─ ulv, klm -> uvkm
│     ├─ ulv
│     └─ klm
└─ vnw, mno -> vwmo
   ├─ vnw
   └─ mno
)

In [21]:
contraction_complexity(optimized_eincode, uniformsize(optimized_eincode, 2))

Time complexity (number of element-wise multiplications) = 2^9.247927513443585
Space complexity (number of elements in the largest intermediate tensor) = 2^4.0
Read-write complexity (number of element-wise read and write) = 2^9.0

In [25]:
# when n = 2, compare the result by OMEinsum and my result
2^9.247927513443585 ≈ 19 * 2^5

true